# Dynamic Web scraper

This notebook scrapes the MSU courses website. 

In [1]:
#Load selinimum and automatically install the Chrome Driver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import FirefoxOptions
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import numpy as np
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from getpass import getpass

from Login import *

In [2]:
driver = get_driver('Chrome') # This might takes a while to boot up
time.sleep(3)
wait = WebDriverWait(driver, 20)
login_to_SIS(driver, authenticate="Okta")
semesters_list, previous_semesters, current_semesters = get_semesters_list(driver)

Please enter your MSU email: 


 danganh1


Please enter your password: 


 ········


Please enter your Okta Verify code: 


 814526


In [3]:
def add_info(driver, element):
    '''Function to add info from the breakout windows for each course
    '''
    print(element.get_attribute('id'))
    driver.execute_script("arguments[0].click();", element) # Click on cell
    # wait.until(EC.frame_to_be_available_and_switch_to_it(0)) 
    iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'iframe'))) # Switch to Class Description frame
    driver.switch_to.frame(iframe)
    wait.until(EC.element_to_be_clickable((By.ID, "MSU_CLS_DTL_WK2_SSR_CL_DTLS_LFF$5$_LBL"))) # Wait until Availability is clickable
     
    schedule_ls = []
    for i in range(3): # Find the class schedules, there might be multiple
        try:
            div = driver.find_element(By.XPATH, f"//div[@id='win9divMSU_CLS_DTL_WK2_HTMLAREA1$160$${i}']")
        except:
            break
        schedule_ls.append(div)
        
    date_tds = driver.find_elements(By.XPATH, "//td[@class='ps_grid-cell E_HTMLAREA2']") # Find class dates
    loc_ins_tds = driver.find_elements(By.XPATH, "//td[@class='ps_grid-cell E_HTMLAREA3']") # Find class locations, instructors, and modes
    
    schedules = []
    dates = []
    locs = []
    modes = []
    names = []
    emails = []
    for i in range(len(loc_ins_tds)):
        # Append multiple schedules to a list
        schedules.append(schedule_ls[i].text)
        
        # Append multiple dates to a list
        dates.append(date_tds[i].text)

        # Append multiple locations and modes to lists
        locs.append(loc_ins_tds[i].text.split('\n')[0])
        modes.append(loc_ins_tds[i].text.split('\n')[-1])

        # Append multiple professors in different schedules to lists
        a_elements = loc_ins_tds[i].find_elements(By.TAG_NAME, "a")
        if not a_elements:
            continue
        email_ind = ''
        name_ind = ''
        for a_element in a_elements: # If one schedule of the class has more than 1 professor, separate with '\n'
            # Extract the email address from the href attribute
            email_ind += a_element.get_attribute('href').split(':')[1] + '\n'
            name_ind += a_element.text + '\n'
        emails.append(email_ind.strip())
        names.append(name_ind.strip())

    element = driver.find_element(By.ID, "MSU_CLS_DTL_WK2_SSR_CL_DTLS_LFF$5$_LBL") 
    element.click() # Click on Availability 
    wait.until(EC.visibility_of_element_located((By.ID, "win9divMSU_CLS_DTL_WK2_HTMLAREA6$22$$0")))

    avails = []
    for i in range(10):
        try: 
            element = driver.find_element(By.ID, f"win9divMSU_CLS_DTL_WK2_HTMLAREA6$22$${i}")
            avails.append(element.text)
        except: 
            break
    
    if len(avails) > 1:
        enrolled = avails.pop()
        limit = sum([int(x.split('/')[-1]) for x in avails])
        avails = [enrolled + '/' + str(limit) + '*']
    
    cancel_cmd="javascript:doUpdateParent(document.win9,'#ICCancel');" 
    driver.execute_script(cancel_cmd); # Close the frame
    driver.switch_to.default_content(); # Switch to the main page
    wait.until(EC.staleness_of(iframe))
    return [schedules, dates, locs, modes, names, emails, avails[0]]

In [4]:
def get_info(driver):
    '''Scrape entire page for all of the class boxes'''
    try:
        per_page = int(driver.find_element(By.ID, 'MSU_RSLT_NAV_WK_NUM2').text)
        result_element = driver.find_element(By.ID, 'MSU_RSLT_NAV_WK_PTPG_ROWS_GRID').text
        result = int(result_element.split()[0])
        pages = (result + per_page - 1) // per_page # get the number of aggregated pages
    except:
        pages = 1
        
    reshaped_list = []
    counter = 1
    while True:
        cells = driver.find_elements(By.XPATH, "//tr[@class='ps_grid-row psc_rowact']") # Find all table row in the page (Classes)
        for cell in cells:
            values = cell.text.split("\n") # Split text in a cell
            if values[1].split(')')[-1] == '':
                removed = values.pop(2)
                values[1] += removed
            values = values[:3] + add_info(driver, cell) # Get all other info
            reshaped_list.append(values) 
        if counter == pages:
            break
        url = f"javascript:submitAction_win9(document.win9,'MSU_RSLT_NAV_WK_SEARCH_CONDITION2$46$');"
        driver.execute_script(url) # If there's more pages, click Next
        time.sleep(3)
        print("Next page")
        counter += 1
    
    col_names = ['Course', 'Type', 'Section', 'Schedule', 'Dates', 'Location', 
                 'Mode', 'Instructor', 'Email', 'Availability']
    df = pd.DataFrame(reshaped_list, columns=col_names)
    return df

In [5]:
def reformat_courses_df(df): 
    # Explode all these columns, will result in duplicate rows for classes that have multiple schedules
    try: 
        df = df.explode(['Schedule', 'Dates', 'Location', 'Mode', 'Email', 'Instructor'])
    except: # In cases every class doesn't have an instructor
        df = df.explode(['Schedule', 'Dates', 'Location', 'Mode'])
        df = df.explode(['Email', 'Instructor'])
    try: 
        df[['Days', 'Time']] = df['Schedule'].str.split(':',n=1,expand=True)
    except: # In case every classes' schedule is 'To Be Announced'
        df[['Days', 'Time']] = np.nan, np.nan
    df[['Course Code', 'Course Name']] = df['Course'].str.split(':', n=1, expand=True)  
    split_result = df['Type'].str.split('(', n=1, expand=True)
    # Check if the split operation resulted in two columns
    if len(split_result.columns) == 2:
        df[['Type', 'Units']] = split_result
    else:
        # Handle the case where the split didn't result in two columns
        df['Type'] = split_result[0]  # Assign the first part to 'Type'
        df['Units'] = '' 
    df[['Section', 'Class Nbr', 'Academic Session']] = df['Section'].str.split('/', n=2, expand=True)
    df[['Units','Status']] = df['Units'].str.split(')', n=1, expand=True)
    df[['Subject','Course Number']] = df['Course Code'].str.split(' ',n=1,expand=True)
    df['Dates'] = df['Dates'].apply(lambda x: x.replace("Approval Required", '').strip() if not pd.isna(x) else x)
    df['Status'] = df['Status'].str.replace('Reserved Capacity', '').str.strip()
    
    df['Course Name'] = df['Course Name'].apply(lambda x: x.replace('Approval Required', '') if not pd.isna(x) else x)
    df['Course Name'] = df['Course Name'].apply(lambda x: x.replace('Cross-Listed', '') if not pd.isna(x) else x)

    df = df.drop(['Course', 'Schedule','Course Code'], axis=1)
    df = df[['Subject','Course Number','Course Name','Type','Units','Status',
             'Section','Class Nbr','Academic Session','Days','Time','Dates', 
             'Location', 'Mode', 'Email', 'Instructor', 'Availability']]
    df['Units'] = df['Units'].str.replace(' units', '')
    df['Section'] = df['Section'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Class Nbr'] = df['Class Nbr'].str.extract(r'(\d+(?:\.\d+)?)')

    df = df.map(lambda x: np.nan if not pd.isna(x) and 'Instructor:' in x else x)
    df.replace(['To Be Announced', 'Arranged', 'No room - Prof will arrange', ' -'], np.nan, inplace=True)
    df = df.groupby(df.index).agg(lambda x: list(x)) # Combining rows with same index into list
    df = df.map(lambda ls: [x for x in ls if not pd.isna(x)] if len(ls) > 1 else ls) # Dropping pd.NA in all lists in row 
    df = df.map(lambda ls: ls[0] if len(ls) == 1 or len(set(ls)) == 1 else ls) # Getting rid of redundancy in lists in row
    df = df.map(lambda x: np.nan if x == [] else x)
    for index, row in df.iterrows():
        if isinstance(row.Days, list) and not isinstance(row.Time, list):
            row.Days = ''.join(row.Days)
    return df

In [6]:
def scrape_by_semester(driver, search_semester, subject):
    statement = switch_to_semester(driver, search_semester, previous_semesters)
    if statement == "No need to switch Semester":
        element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_CLEAR_FLDS_PB') # Click New Search
        driver.execute_script("arguments[0].click();", element)
        time.sleep(3)
    element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_SUBJECT')  
    element.send_keys(subject) #pick cmse for example
    url = f"javascript:submitAction_win9(document.win9,'MSU_CLSRCH_WRK_SSR_PB_SEARCH');"
    driver.execute_script(url); # Hit search
    wait.until(EC.element_to_be_clickable((By.ID, 'DESCR100$0_row_0')))
        
    df = get_info(driver)
    df = reformat_courses_df(df)
    return df

df = scrape_by_semester(driver, 'Spring Semester 2024', "WRA")
df

DESCR100$0_row_0
DESCR100$0_row_1
DESCR100$0_row_2
DESCR100$0_row_3
DESCR100$0_row_4
DESCR100$0_row_5
DESCR100$0_row_6
DESCR100$0_row_7
DESCR100$0_row_8
DESCR100$0_row_9
DESCR100$0_row_10
DESCR100$0_row_11
DESCR100$0_row_12
DESCR100$0_row_13
DESCR100$0_row_14
DESCR100$0_row_15
DESCR100$0_row_16
DESCR100$0_row_17
DESCR100$0_row_18
DESCR100$0_row_19
DESCR100$0_row_20
DESCR100$0_row_21
DESCR100$0_row_22
DESCR100$0_row_23
DESCR100$0_row_24
DESCR100$0_row_25
DESCR100$0_row_26
DESCR100$0_row_27
DESCR100$0_row_28
DESCR100$0_row_29
DESCR100$0_row_30
DESCR100$0_row_31
DESCR100$0_row_32
DESCR100$0_row_33
DESCR100$0_row_34
DESCR100$0_row_35
DESCR100$0_row_36
DESCR100$0_row_37
DESCR100$0_row_38
DESCR100$0_row_39
DESCR100$0_row_40
DESCR100$0_row_41
DESCR100$0_row_42
DESCR100$0_row_43
DESCR100$0_row_44
DESCR100$0_row_45
DESCR100$0_row_46
DESCR100$0_row_47
DESCR100$0_row_48
DESCR100$0_row_49
Next page
DESCR100$0_row_0
DESCR100$0_row_1
DESCR100$0_row_2
DESCR100$0_row_3
DESCR100$0_row_4
DESCR100$0_row_

,Subject,Course Number,Course Name,Type,Units,Status,Section,Class Nbr,Academic Session,Days,Time,Dates,Location,Mode,Email,Instructor,Availability
0,WRA,0102,Preparation for College Writing,Laboratory,0.00,Open,007,11638,Regular Academic Session,Fri,12:40 PM-2:30 PM,1/8/2024 - 4/21/2024,NaN,Online Synchronous,xqli@msu.edu,Xinqiang Li,2/24
1,WRA,1004,Preparation for College Writing,Lecture,3.00,Open,007,11641,Regular Academic Session,Mon Wed,3:00 PM-4:20 PM,1/8/2024 - 4/21/2024,NaN,Online Synchronous,xqli@msu.edu\nlindqu11@msu.edu,Xinqiang Li\nJulie Lindquist,2/24
2,WRA,1004,Preparation for College Writing,Lecture,3.00,Open,007,11641,Regular Academic Session,Mon Wed,3:00 PM-4:20 PM,1/8/2024 - 4/21/2024,NaN,Online Synchronous,xqli@msu.edu\nlindqu11@msu.edu,Xinqiang Li\nJulie Lindquist,2/24
3,WRA,101,Writing as Inquiry,Lecture,4.00,Open,001,11644,Regular Academic Session,Mon Wed,8:00 AM-9:50 AM,1/8/2024 - 4/21/2024,Ernst Bessey Hall 312,In Person,kangmi23@msu.edu,Stephie Kang,23/24
4,WRA,101,Writing as Inquiry,Lecture,4.00,Wait List,002,11645,Regular Academic Session,Mon Wed,8:00 AM-9:50 AM,1/8/2024 - 4/21/2024,Ernst Bessey Hall 107,In Person,caesarc@msu.edu,Cheryl Caesar,24/24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,WRA,890,Independent Study in Rhetoric and Writing,Lecture,1.00 - 3.00,Wait List,005,37644,Regular Academic Session,NaN,None,1/8/2024 - 4/21/2024,NaN,In Person,devossda@msu.edu,Danielle Nicole Devoss,1/1
245,WRA,890,Independent Study in Rhetoric and Writing,Lecture,1.00 - 3.00,Wait List,006,37732,Regular Academic Session,NaN,None,1/8/2024 - 4/21/2024,NaN,In Person,smit1254@msu.edu,Trixie Smith,1/1
246,WRA,893B,Internship in Professional Writing,Lecture,3.00,Open,001,11810,Regular Academic Session,NaN,None,1/8/2024 - 4/21/2024,NaN,In Person,bstraay@msu.edu,Bree Straayer,1/10
247,WRA,899,Master's Thesis Research,Lecture,1.00 - 3.00,Open,001,11811,Regular Academic Session,NaN,None,1/8/2024 - 4/21/2024,NaN,In Person,lpotts@msu.edu,Liza Potts,1/10


In [7]:
df.to_csv('WRA.csv', index=False)